In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-laakd1a0
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-laakd1a0
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=8b275d4dcbc577de9856f874de91d6d0d3490406b1272c2a519f90c01f595cee
  Stored in directory: /tmp/pip-ephem-wheel-cache-q03jll6o/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name conv_host.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <ctime>
#include <iostream>

#include <cuda.h>
#include <cuda_runtime.h>

using namespace std;

int isvalid(int i, int j, int n)
{
    if(i < 0 || j < 0 || i >= n || j >= n)
        return 0;
    return 1;
}

void conv2Dcpu(float *N, float *P, int n, int m)
{
    int k = m/2; // m = 2*k + 1
    float psum = 0;
 
    for(int i=0; i<n; i++)
    {
        for(int j=0; j<n; j++)
        {
            psum = 0;
            for(int p = i-k; p <= i+k; p++)
            {
                for(int q = j-k; q <= j+k; q++)
                {
                    if(isvalid(p, q, n))
                        psum += N[n*p + q];
                }
            }
            P[n*i + j] = psum / float(m*m);
        }
    }
}

void print_matrix(float *a, int n, char c)
{
    if(c == 'N')
        cout<<"Printing source matrix N = "<<endl;
    else
        cout<<"Printing convolved matrix P = "<<endl;
    for(int i=0; i<n; i++)
    {
        for(int j=0; j<n; j++)
            printf("%.2f ", a[n*i + j]);
        cout<<endl;
    }
    cout<<endl;
}

int verify_result(float *P, float *h_P, int n)
{
    for(int i=0; i<n; i++)
    {
        for(int j=0; j<n; j++)
        {
            if(abs(P[n*i + j] - h_P[n*i + j]) > 1e-6)
            {
                cout<<"P val = "<<P[n*i + j]<<endl;
                cout<<"h_P val = "<<h_P[n*i + j]<<endl;
                return 0;
            }
        }
    }
    return 1;
}

__global__ void conv2Dbasic(float *d_N, float *d_P, int n, int m)
{
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;
    int k = m/2;
 
    int startrow = i-k;
    int startcol = j-k;
    int row, col;
    float psum = 0;

    if(i < n && j < n)
    {
        for(int r = 0; r < m; r++)
        {
            for(int c = 0; c < m; c++)
            {
                row = startrow + r;
                col = startcol + c;
                if(row >= 0 && col >= 0 && row < n && col < n)
                    psum += d_N[n*row + col];
                
            }
        }
        d_P[n*i + j] = psum / float(m*m);
    }
}

#define TILE_WIDTH 32
#define MAX_MASK_WIDTH 15

__global__ void conv2Dtiled(float *d_N, float *d_P, int n, int m)
{
    int by = blockIdx.y; int bx = blockIdx.x;
    int ty = threadIdx.y; int tx = threadIdx.x;
    int i = by * blockDim.y + ty; //Note that here, blockDim.y = blockDim.x = TILE_WIDTH
    int j = bx * blockDim.x + tx;
    int k = m/2, halo_i, halo_j;
 
    __shared__ float Ns[TILE_WIDTH + MAX_MASK_WIDTH - 1][TILE_WIDTH + MAX_MASK_WIDTH - 1];
 
    //Load k*k elements from upper left corner
    halo_i = (by-1) * blockDim.y + ty;
    halo_j = (bx-1) * blockDim.x + tx;
    if(ty >= TILE_WIDTH - k && tx >= TILE_WIDTH - k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[ty - (blockDim.y - k)][tx - (blockDim.x - k)] = 0;
        else
            Ns[ty - (blockDim.y - k)][tx - (blockDim.x - k)] = d_N[n*halo_i + halo_j];
    }
 
    //Load TW*k elements from upper block
    halo_i = (by-1) * blockDim.y + ty;
    halo_j = bx * blockDim.x + tx;
    if(ty >= TILE_WIDTH - k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[ty - (blockDim.y - k)][k + tx] = 0;
        else
            Ns[ty - (blockDim.y - k)][k + tx] = d_N[n*halo_i + halo_j];
    }
 
    //Load k*k elements from upper right corner
    halo_i = (by-1) * blockDim.y + ty;
    halo_j = (bx+1) * blockDim.x + tx;
    if(ty >= TILE_WIDTH - k && tx < k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[ty - (blockDim.y - k)][k + TILE_WIDTH + tx] = 0;
        else
            Ns[ty - (blockDim.y - k)][k + TILE_WIDTH + tx] = d_N[n*halo_i + halo_j];
    }
 
    //Load k*TW elements from left
    halo_i = by * blockDim.y + ty;
    halo_j = (bx-1) * blockDim.x + tx;
    if(tx >= TILE_WIDTH - k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[k + ty][tx - (blockDim.x - k)] = 0;
        else
            Ns[k + ty][tx - (blockDim.x - k)] = d_N[n*halo_i + halo_j];
    }
 
    //Load internal TW*TW elements, not halo elements
    halo_i = by * blockDim.y + ty;
    halo_j = bx * blockDim.x + tx;

    Ns[k + ty][k + tx] = d_N[n*halo_i + halo_j];
 
    //Load k*TW elements from right
    halo_i = by * blockDim.y + ty;
    halo_j = (bx+1) * blockDim.x + tx;
    if(tx < k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[k + ty][k + TILE_WIDTH + tx] = 0;
        else
            Ns[k + ty][k + TILE_WIDTH + tx] = d_N[n*halo_i + halo_j];
    }
 
    //Load k*k elements from down left corner
    halo_i = (by+1) * blockDim.y + ty;
    halo_j = (bx-1) * blockDim.x + tx;
    if(ty < k && tx >= TILE_WIDTH - k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[k + ty + TILE_WIDTH][tx - (blockDim.x - k)] = 0;
        else
            Ns[k + ty + TILE_WIDTH][tx - (blockDim.x - k)] = d_N[n*halo_i + halo_j];
    }
 
    //Load TW*k elements from down block
    halo_i = (by+1) * blockDim.y + ty;
    halo_j = bx * blockDim.x + tx;
    if(ty < k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[k + ty + TILE_WIDTH][tx + k] = 0;
        else
            Ns[k + ty + TILE_WIDTH][tx + k] = d_N[n*halo_i + halo_j];
    }
 
    //Load k*k elements from down right corner
    halo_i = (by+1) * blockDim.y + ty;
    halo_j = (bx+1) * blockDim.x + tx;
    if(ty < k && tx < k)
    {
        if(halo_i < 0 || halo_j < 0 || halo_i >= n || halo_j >= n)
            Ns[k + ty + TILE_WIDTH][k + tx + TILE_WIDTH] = 0;
        else
            Ns[k + ty + TILE_WIDTH][k + tx + TILE_WIDTH] = d_N[n*halo_i + halo_j];
    }
 
    __syncthreads();
 
    float psum = 0.0;
    if(i<n && j<n)
    {
        for(int r=0; r<m; r++)
        {
            for(int c=0; c<m; c++)
            {
                psum += Ns[ty + r][tx + c];
            }
        }
      
        d_P[n*i + j] = psum / float(m*m);
        //printf("p[i = %d, j = %d] = %.2f\n", i, j, psum/float(m*m));
    }
}

int main(int argc, char* argv[])
{
    int n, m;
    n = atoi(argv[1]); //square matrix width
    m = atoi(argv[2]); //square mask_width, odd number
 
    float *h_N, *h_P, *d_N, *d_P;; //for GPU computations
    float *N, *P; //for CPU computations
 
    //Allocate Memory to cpu variables
    h_N = (float *)malloc(n * n * sizeof(float));
    h_P = (float *)malloc(n * n * sizeof(float));
    N = (float *)malloc(n * n * sizeof(float));
    P = (float *)malloc(n * n * sizeof(float));
 
    //Allocate Memory to gpu variables
    cudaMalloc((void **)&d_N, n * n * sizeof(float));
    cudaMalloc((void **)&d_P, n * n * sizeof(float));
 
    for(int i=0; i<n; i++)
    {
        for(int j=0; j<n; j++)
        {
            h_N[n*i + j] = (float(rand()) / (float(RAND_MAX)));
            //h_N[n*i + j] = 3.0;
            N[n*i + j] = h_N[n*i + j];
        }
    }
    
    //CPU convolution time
    clock_t t1, t2;  
    t1 = clock();
    conv2Dcpu(N, P, n, m);
    t2 = clock();
    print_matrix(N, n, 'N');
    print_matrix(P, n, 'P');
    printf("+++ n = %d, CPU Time taken = %lf ms\n", n, ((double)(t2-t1)/(double)CLOCKS_PER_SEC) * 1000);
 
    //GPU convolution basic
    cudaMemcpy(d_N, h_N, n * n *sizeof(float), cudaMemcpyHostToDevice);
    float grid_dim = ceil(sqrt(n*n / 1024.0));
    //cout<<"grid dim = "<<grid_dim<<endl;
    dim3 grid(int(grid_dim), int(grid_dim), 1);
    dim3 block(32,32,1);
    float ms;

    cudaEvent_t c1, c2;
    cudaEventCreate(&c1);
    cudaEventCreate(&c2);
 
    cudaEventRecord(c1);
    conv2Dbasic<<<grid, block>>>(d_N, d_P, n, m);
    cudaEventRecord(c2);
 
    cudaEventSynchronize(c2);
    cudaEventElapsedTime(&ms, c1, c2);

    cudaMemcpy(h_P, d_P, n * n *sizeof(float), cudaMemcpyDeviceToHost);
    print_matrix(h_P, n, 'P');
    if(verify_result(P, h_P, n))
        printf("+++ n = %d, GPU basic Time taken = %lf ms\n", n, ms);
 
    //Reallocate used placeholders
    cudaFree(d_P);
    free(h_P);
    cudaMalloc((void **)&d_P, n * n * sizeof(float));
    h_P = (float *)malloc(n * n * sizeof(float));
 
    //GPU convolution tiled
    cudaEvent_t g1, g2;
    cudaEventCreate(&g1);
    cudaEventCreate(&g2);
 
    cudaEventRecord(g1);
    conv2Dtiled<<<grid, block>>>(d_N, d_P, n, m);
    cudaEventRecord(g2);
    cudaEventSynchronize(g2);
    cudaEventElapsedTime(&ms, g1, g2);

    cudaMemcpy(h_P, d_P, n * n *sizeof(float), cudaMemcpyDeviceToHost);
    print_matrix(h_P, n, 'P');
    if(verify_result(P, h_P, n))
        printf("+++ n = %d, GPU tiled Time taken = %lf ms\n", n, ms);
 
    cudaFree(d_N);
    cudaFree(d_P);
    free(h_P);
    free(h_N);
    free(N);
    free(P);

    return 0;
}

'File written in /content/src/conv_host.cu'

In [ ]:
!nvcc /content/src/conv_host.cu -o /content/src/conv_host

In [ ]:
!/content/src/conv_host 8 3

Printing source matrix N = 
0.84 0.39 0.78 0.80 0.91 0.20 0.34 0.77 
0.28 0.55 0.48 0.63 0.36 0.51 0.95 0.92 
0.64 0.72 0.14 0.61 0.02 0.24 0.14 0.80 
0.16 0.40 0.13 0.11 1.00 0.22 0.51 0.84 
0.61 0.30 0.64 0.52 0.49 0.97 0.29 0.77 
0.53 0.77 0.40 0.89 0.28 0.35 0.81 0.92 
0.07 0.95 0.53 0.09 0.19 0.66 0.89 0.35 
0.06 0.02 0.46 0.06 0.24 0.97 0.90 0.85 

Printing convolved matrix P = 
0.23 0.37 0.40 0.44 0.38 0.36 0.41 0.33 
0.38 0.54 0.57 0.53 0.48 0.41 0.54 0.43 
0.30 0.39 0.42 0.39 0.41 0.44 0.57 0.46 
0.31 0.41 0.40 0.41 0.46 0.43 0.53 0.37 
0.31 0.44 0.46 0.50 0.54 0.55 0.63 0.46 
0.36 0.53 0.56 0.45 0.50 0.55 0.67 0.45 
0.27 0.42 0.46 0.35 0.42 0.59 0.75 0.52 
0.12 0.23 0.23 0.17 0.25 0.43 0.51 0.33 

+++ n = 8, CPU Time taken = 0.006000 ms
Printing convolved matrix P = 
0.23 0.37 0.40 0.44 0.38 0.36 0.41 0.33 
0.38 0.54 0.57 0.53 0.48 0.41 0.54 0.43 
0.30 0.39 0.42 0.39 0.41 0.44 0.57 0.46 
0.31 0.41 0.40 0.41 0.46 0.43 0.53 0.37 
0.31 0.44 0.46 0.50 0.54 0.55 0.63 0.46 
0.36 0.

In [ ]:
!cuda-memcheck /content/src/conv_host 64 3

========= CUDA-MEMCHECK
Printing source matrix N = 
0.84 0.39 0.78 0.80 0.91 0.20 0.34 0.77 0.28 0.55 0.48 0.63 0.36 0.51 0.95 0.92 0.64 0.72 0.14 0.61 0.02 0.24 0.14 0.80 0.16 0.40 0.13 0.11 1.00 0.22 0.51 0.84 0.61 0.30 0.64 0.52 0.49 0.97 0.29 0.77 0.53 0.77 0.40 0.89 0.28 0.35 0.81 0.92 0.07 0.95 0.53 0.09 0.19 0.66 0.89 0.35 0.06 0.02 0.46 0.06 0.24 0.97 0.90 0.85 
0.27 0.54 0.38 0.76 0.51 0.67 0.53 0.04 0.44 0.93 0.93 0.72 0.28 0.74 0.64 0.35 0.69 0.17 0.44 0.88 0.83 0.33 0.23 0.89 0.35 0.69 0.96 0.59 0.66 0.86 0.44 0.92 0.40 0.81 0.68 0.91 0.48 0.22 0.95 0.92 0.15 0.88 0.64 0.43 0.62 0.28 0.79 0.31 0.45 0.23 0.19 0.28 0.56 0.42 0.17 0.91 0.10 0.13 0.50 0.76 0.98 0.94 0.68 0.38 
0.75 0.37 0.29 0.23 0.58 0.24 0.15 0.73 0.13 0.79 0.16 0.75 0.07 0.95 0.05 0.52 0.18 0.24 0.80 0.73 0.66 0.97 0.64 0.76 0.09 0.13 0.52 0.08 0.07 0.20 0.46 0.82 0.57 0.76 0.05 0.16 1.00 0.20 0.89 0.13 1.00 0.05 0.87 0.07 0.00 0.92 0.59 0.18 0.16 0.39 0.91 0.82 0.36 0.55 0.58 0.45 0.69 0.10 0.53 0.76 0.30 0

In [ ]:
!/content/src/conv_host 32 3

+++ n = 32, CPU Time taken = 0.066000 ms
+++ n = 32, GPU basic Time taken = 0.024384 ms
+++ n = 32, GPU tiled Time taken = 0.016672 ms


In [ ]:
!cuda-memcheck /content/src/conv_host 8192 15

========= CUDA-MEMCHECK
+++ n = 8192, CPU Time taken = 84177.104000 ms
+++ n = 8192, GPU basic Time taken = 6739.307617 ms
+++ n = 8192, GPU tiled Time taken = 2129.368896 ms
========= ERROR SUMMARY: 0 errors


In [ ]:
!nvprof --events warps_launched,local_load --metrics all /content/src/conv_host 4096 15

==290== NVPROF is profiling process 290, command: /content/src/conv_host 4096 15
+++ n = 4096, CPU Time taken = 21143.414000 ms
==290== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==290== Replaying kernel "conv2Dbasic(float*, float*, int, int)" (1 of 60)... 
	4 internal events
==290== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (2 of 60)... 
	4 internal events
==290== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (3 of 60)... 
	global_load
	l2_subp0_write_tex_hit_sectors
	l2_subp1_write_tex_hit_sectors
	7 internal events
==290== 





Replaying kernel "conv2Dbasic(float*, float*, int, int)" (4 of 60)... 
	4 internal events
==290== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (5 of 60)... 
	2 internal events
==290== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (6 of 60)... 
	2 internal events
==290== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (7 of 60)... 
	4 internal eve

======== Error: no application specified.


In [ ]:
!/content/src/conv_host 5 3

+++ n = 5, CPU Time taken = 0.004000 ms
Printing convolved matrix P = 
1.33 2.00 2.00 2.00 1.33 
2.00 3.00 3.00 3.00 2.00 
2.00 3.00 3.00 3.00 2.00 
2.00 3.00 3.00 3.00 2.00 
1.33 2.00 2.00 2.00 1.33 

+++ n = 5, GPU basic Time taken = 0.022528 ms
Printing convolved matrix P = 
1.33 2.00 2.00 2.00 2.00 
2.00 3.00 3.00 3.00 3.00 
2.00 3.00 3.00 3.00 3.00 
2.00 3.00 3.00 3.00 2.67 
1.33 2.00 2.00 2.00 1.67 

P val = 1.33333
h_P val = 2


In [ ]:
!/content/src/conv_host 512 3

+++ n = 512, CPU Time taken = 20.250000 ms
+++ n = 512, GPU basic Time taken = 0.037600 ms
+++ n = 512, GPU tiled Time taken = 0.055872 ms


In [ ]:
!/content/src/conv_host 1024 3

+++ n = 1024, CPU Time taken = 63.835000 ms
+++ n = 1024, GPU basic Time taken = 0.100992 ms
+++ n = 1024, GPU tiled Time taken = 0.114304 ms


In [ ]:
!/content/src/conv_host 2048 3

+++ n = 2048, CPU Time taken = 268.506000 ms
+++ n = 2048, GPU basic Time taken = 0.364992 ms
+++ n = 2048, GPU tiled Time taken = 0.401216 ms


In [ ]:
!/content/src/conv_host 4096 3

+++ n = 4096, CPU Time taken = 1003.326000 ms
+++ n = 4096, GPU basic Time taken = 1.428384 ms
+++ n = 4096, GPU tiled Time taken = 1.524800 ms


In [ ]:
!/content/src/conv_host 512 7

+++ n = 512, CPU Time taken = 75.382000 ms
+++ n = 512, GPU basic Time taken = 0.080992 ms
+++ n = 512, GPU tiled Time taken = 0.068032 ms


In [ ]:
!/content/src/conv_host 1024 7

+++ n = 1024, CPU Time taken = 304.779000 ms
+++ n = 1024, GPU basic Time taken = 0.283872 ms
+++ n = 1024, GPU tiled Time taken = 0.223744 ms


In [ ]:
!/content/src/conv_host 2048 7

+++ n = 2048, CPU Time taken = 1202.000000 ms
+++ n = 2048, GPU basic Time taken = 1.061600 ms
+++ n = 2048, GPU tiled Time taken = 0.815296 ms


In [ ]:
!/content/src/conv_host 4096 7

+++ n = 4096, CPU Time taken = 4851.096000 ms
+++ n = 4096, GPU basic Time taken = 4.203872 ms
+++ n = 4096, GPU tiled Time taken = 3.175456 ms


In [ ]:
!/content/src/conv_host 8192 7

+++ n = 8192, CPU Time taken = 19365.587000 ms
+++ n = 8192, GPU basic Time taken = 16.768192 ms
+++ n = 8192, GPU tiled Time taken = 12.597152 ms


In [ ]:
!/content/src/conv_host 2048 15

+++ n = 2048, CPU Time taken = 5266.374000 ms
+++ n = 2048, GPU basic Time taken = 2.968096 ms
+++ n = 2048, GPU tiled Time taken = 1.655712 ms


In [ ]:
!/content/src/conv_host 4096 15

+++ n = 4096, CPU Time taken = 22172.030000 ms
+++ n = 4096, GPU basic Time taken = 11.779264 ms
+++ n = 4096, GPU tiled Time taken = 6.534272 ms


In [ ]:
!/content/src/conv_host 8192 15

+++ n = 8192, CPU Time taken = 86789.802000 ms
+++ n = 8192, GPU basic Time taken = 116.821152 ms
+++ n = 8192, GPU tiled Time taken = 44.108894 ms


In [ ]:
!nvidia-smi

Sun Apr 12 04:05:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
!/content/src/conv_host 512 3

/bin/bash: /content/src/conv_host: No such file or directory


In [ ]:
!nvprof --events warps_launched,local_load --metrics all /content/src/conv_host 4096 15

==261== NVPROF is profiling process 261, command: /content/src/conv_host 4096 15
+++ n = 4096, CPU Time taken = 21383.277000 ms
==261== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==261== Replaying kernel "conv2Dbasic(float*, float*, int, int)" (1 of 55)... 
	4 internal events
==261== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (2 of 55)... 
	active_cycles
	shared_load
	9 internal events
==261== 




Replaying kernel "conv2Dbasic(float*, float*, int, int)" (3 of 55)... 
	4 internal events
==261== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (4 of 55)... 
	4 internal events
==261== 


Replaying kernel "conv2Dbasic(float*, float*, int, int)" (5 of 55)... 
	elapsed_cycles_sm
	inst_executed_shared_atom_cas
	shared_atom
	inst_issued1
	inst_issued2
	l2_subp0_read_tex_hit_sectors
	l2_subp1_read_tex_hit_sectors
	4 internal events
==261== 









Replaying kernel "conv2Dbasic(float*, float*, int, int)" (6 of 55)... 
	1 int